In [192]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import gzip, json
from Bio import SeqIO
import scipy.sparse as sp
from collections import Counter
import numpy as np
import pandas as pd
fasta_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/reads.fasta.gz'
paf_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/alignment.paf.gz'
with gzip.open(paf_gz_file, "rt") as file:
    max_values = {}  
    for row in file:  
        columns = row.strip().split('\t') 
        query_id = columns[0]  
        match_bases = int(columns[9]) 
        max_values[query_id] = columns 
        if query_id in max_values:  
            if match_bases > int(max_values[query_id][9]):  
                max_values[match_bases] = columns
        else:  
            continue

In [193]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
import seaborn
import sharedmem
import pygraphviz

In [203]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from truth import get_overlaps
from evaluate import NearestNeighborsConfig, mp_compute_nearest_neighbors
from plots import plot_read_graph, mp_plot_read_graphs, get_graphviz_layout, get_umap_layout

/home/miaocj/docker_dir/kNN-overlap-finder/scripts/../lib


In [194]:
from memory_profiler import profile
%load_ext memory_profiler


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [195]:
%memit x = 10+5

peak memory: 5395.32 MiB, increment: 0.00 MiB


In [196]:
MAX_SAMPLE_SIZE = int(1e9)
COVERAGE_DEPTH = 20
max_n_neighbors = 20
with gzip.open(fasta_gz_file, "rt") as handle:
    flag = 0
    seq_num = 0
    aligned_num = 0
    for record in SeqIO.parse(handle, "fasta"):
        seq_num += 1
        if record.id in max_values.keys():
            aligned_num+=1
            columns = max_values[record.id]
            if int(columns[9])/int(columns[1]) > 0.5:
                flag += 1
    print(flag/seq_num)
    print(aligned_num/seq_num)

0.9493518828919433
0.9829031176667784


In [197]:
npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_30k/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_30k//kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_30k/kmer_k16/read_features.json.gz"

nbr_path = '/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/Ecoli/all/ONT/kmer_k16/HNSW_Euclidean_scBiMap_500d_TF-IDF_nbr_matrix.npz'
time_path = '/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/Ecoli/all/ONT/kmer_k16/HNSW_Euclidean_scBiMap_500d_TF-IDF_time_usage.json'
meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

In [198]:
feature_matrix.shape

(7448, 1265400)

In [ ]:
import numpy as np  
import collections  
from typing import Mapping  
from numba import jit


def _hash(kmer_index: str, seed: int) -> int:
    hash_value = mmh3.hash(kmer_index, seed=seed)
    binary_string = format(hash_value & 0xFFFFFFFF, '032b')
    hash_list = [int(x) for x in binary_string]
    hash_list1 = np.array(hash_list)
    return hash_list1

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]):  
            kmer_index_str = str(kmer_index)  
            hash_list = _hash(kmer_index_str, seed=s)  
            kmer_hash_indice[kmer_index] = hash_list  
  
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]
    
concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 20, seed = 4829)

299897765
2378963497
44873935
2670060064
25935175
920060685
1586326711
1550303102
278283145
1103905279
2497529993
993549511
3538826777
827898949
1744767203
1454149367
347405931
293704892
739522939
2787976300


In [ ]:
 np.count_nonzero(concat_simhash == 0)/concat_simhash.size 

0.5056349019871106

In [ ]:
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
  
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]

In [200]:
meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

feature_weights = {i: 1 for i in range(feature_matrix.shape[1])}

In [204]:
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graphs[k] = graph

In [205]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

(7448, 150966, 14394, 2)

In [208]:
df_rows = []

for k in k_values:
    graph = graphs[k]
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [209]:
graph_stats

{'precision': 0.9025594824591873,
 'nr_precision': 0.14440778047933311,
 'recall': 0.5507928937641589,
 'nr_recall': 0.9242740030568293,
 'singleton_count': 22,
 'singleton_fraction': 0.002953813104189044,
 'N50': 3717}